In [17]:
import os
import re
import locale
# import pandas as pd
from datetime import datetime as dt
import pandas as pd
# from datetime import date, timedelta
# import matplotlib.pyplot as plt

In [18]:
# Mes del Reporte
mes_reporte = 7

sep = '~'

In [19]:
# Locación
locale.setlocale(locale.LC_ALL, '')

# Rutas
mainpath = os.getcwd()
inputpath = os.path.join(mainpath, "input")
reportspath = os.path.join(mainpath, "reports")


# Momento del reporte
inicio_date = dt.now()
inicio_string = inicio_date.strftime("%Y%m%d_%H%M%S")

# Directorio Reporte
outputpath = os.path.join(reportspath, inicio_string)
concatenadopath = os.path.join(outputpath, "01_Concatenado.txt")
limpiadopath = os.path.join(outputpath, "02_Limpiado.txt")
consolidadopath = os.path.join(outputpath, "03_Consolidado.txt")

# Exporte Excel
excelOutputPath = os.path.join(outputpath, "04_Reporte.xlsx")

# Array Directorios
directorios_requeridos = [inputpath, reportspath, outputpath]


In [20]:
for directorio in directorios_requeridos:
    if not os.path.exists(directorio):
        os.mkdir(directorio)

In [21]:
with open(concatenadopath, "w+") as concatenado:
    for log_error in os.listdir(inputpath):
        logpath = os.path.join(inputpath, log_error)
        with open(logpath) as logs:
            for log in logs:
                concatenado.write(log)

In [22]:
with open(limpiadopath, "w+") as limpiadofile:
    with open(concatenadopath, "r") as concatenado:
        limpiado = re.sub(r'\n', '', concatenado.read())
        limpiado = re.sub(r'(line\s\d+)(\[)', r'\1\n\2', limpiado)
        limpiadofile.write(limpiado)

In [23]:
with open(consolidadopath, "w+") as consolidadopath:
    with open(limpiadopath, "r") as limpiadopath:
        lines = limpiadopath.readlines()
        for i in range(len(lines)):
            try:
                linea = lines[i]

                # Obtener Fecha
                fecha = linea[1:21]
                linea = linea[22:]

                #Obtener Location
                locacion = re.findall(r"(^.*?)\]", linea)[0]
                linea = re.sub(r'(^.*?)\] ', '', linea)

                #Obtener Tipo Error
                tipoError = re.findall(r"(^.*?)\:", linea)[0]
                linea = re.sub(r'(^.*?)\:  ', '', linea)

                #Obtener Ubicacion Archivo
                ubicacion = re.findall(r"in\s(/home/nave.*.php)", linea)[0]
                linea = re.sub(r'in\s(/home/nave.*.php)', '', linea)

                #Obtener Linea Error
                lineError = re.findall(r"on\s(line\s\d+)$", linea)[0]
                linea = re.sub(r'on\s(line\s\d+)$', '', linea)

                registro  = fecha + sep
                registro += locacion + sep
                registro += tipoError + sep
                registro += ubicacion + sep
                registro += lineError + sep
                registro += linea

                consolidadopath.write(registro)
            except:

                print(linea)

In [24]:
dfCore = pd.read_csv(consolidadopath.name, sep=sep, names=["Fecha_Hora", "Localidad", "Tipo_Error", "Ubicacion", "Linea", "Mensaje_Error"])

In [25]:
dfCore['Fecha_Hora'] = pd.to_datetime(dfCore['Fecha_Hora'])

In [26]:
dfCore['Fecha'] = pd.to_datetime(dfCore['Fecha_Hora']).dt.date
dfCore['Hora'] = pd.to_datetime(dfCore['Fecha_Hora']).dt.time

In [27]:
dfCore['Dia'] = pd.to_datetime(dfCore['Fecha_Hora']).dt.day
dfCore['Mes'] = pd.to_datetime(dfCore['Fecha_Hora']).dt.month
dfCore['Año'] = pd.to_datetime(dfCore['Fecha_Hora']).dt.year

In [28]:
dfCore.tail()

,Fecha_Hora,Localidad,Tipo_Error,Ubicacion,Linea,Mensaje_Error,Fecha,Hora,Dia,Mes,Año
526468,2023-07-31 23:59:41,/Chile/Continental,PHP Notice,/home/nave13/clases/bd/DAO/DAOCredencialSesion...,line 268,Undefined property: DAOCredencialSesion::$at_o...,2023-07-31,23:59:41,31,7,2023
526469,2023-07-31 23:59:41,/Chile/Continental,PHP Fatal error,/home/nave13/clases/utiles/webpaywebservice/We...,line 697,Call to a member function getCodigo() on a non...,2023-07-31,23:59:41,31,7,2023
526470,2023-08-01 00:01:04,/Chile/Continental,PHP Notice,/home/nave13/clases/bd/ManejadorSP.php,line 129,"sybase_query(): Sybase: Unexpected results, c...",2023-08-01,00:01:04,1,8,2023
526471,2023-08-01 00:01:04,/Chile/Continental,PHP Notice,/home/nave13/clases/bd/DAO/DAOCredencialSesion...,line 268,Undefined property: DAOCredencialSesion::$at_o...,2023-08-01,00:01:04,1,8,2023
526472,2023-08-01 00:01:04,/Chile/Continental,PHP Fatal error,/home/nave13/clases/utiles/webpaywebservice/We...,line 697,Call to a member function getCodigo() on a non...,2023-08-01,00:01:04,1,8,2023


In [29]:
dfCore = dfCore[(dfCore['Mes'] == mes_reporte)]

In [30]:
dfGroupByError = dfCore.groupby(["Ubicacion", "Tipo_Error"]).size().reset_index(name='counts')

In [31]:
dfGroupByError.sort_values(by='counts', ascending=False)

,Ubicacion,Tipo_Error,counts
90,/home/nave13/administracion/procesos/pagare/co...,PHP Warning,310636
163,/home/nave13/administracion/procesos/solicitud...,PHP Notice,26464
355,/home/nave13/clases/utiles/webpaywebservice/We...,PHP Notice,16718
597,/home/nave13/portaldepagos/funciones/f_finanza...,PHP Warning,10278
352,/home/nave13/clases/bd/ManejadorSP.php,PHP Warning,10130
...,...,...,...
325,/home/nave13/clases/bd/DAO/DAOAsignaturaSancio...,PHP Notice,1
314,/home/nave13/clases/administradores/Administra...,PHP Parse error,1
303,/home/nave13/alumno/solicitudes_academicas/ret...,PHP Notice,1
297,/home/nave13/alumno/solicitudes_academicas/rec...,PHP Notice,1


In [32]:
with pd.ExcelWriter(excelOutputPath) as writer:

    dfCore.to_excel(writer, sheet_name="Data", index=False)
    dfGroupByError.to_excel(writer, sheet_name="Agrupado", index=False)